In [2]:
import os
import pandas as pd
import re
import math
from collections import Counter
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

print("✅ Imports loaded")

✅ Imports loaded


In [ ]:
labels_path = "data/labels"

if not os.path.exists(labels_path):
    raise FileNotFoundError(f"❌ labels file NOT found at: {labels_path}")

labels = []
filepaths = []

with open(labels_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 2:
            label, path = parts
            labels.append(label)
            filepaths.append(path.replace("../", ""))

print("✅ Loaded labels:", len(labels))
print("Sample lines:", list(zip(labels[:5], filepaths[:5])))


FileNotFoundError: ❌ labels file NOT found at: data/labels